# Fase 1: Information Theory - Validación Híbrida Ventanas

**Objetivo**: Calcular Mutual Information entre features diarias y retornos futuros para identificar días con información predictiva.

**Método**: Information Theory (model-agnostic)
- Mutual Information I(X_t; y) por día relativo
- Filtrado rápido: descarta días sin señal
- Solo usa columnas básicas de DIB bars

**Output**: `phase1_results.pkl` con info_results por evento

**Tiempo estimado**: 10-20 min (con sample_size=200)

## 0. Setup

In [ ]:
import polars as pl
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mutual_info_score
import pickle
import warnings
warnings.filterwarnings('ignore')

# Config
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

# Paths
BARS_ROOT = Path('../../../../processed/dib_bars/pilot50_validation')
WATCHLIST = Path('../../../../processed/universe/pilot50_validation/daily')
OUTPUT_DIR = Path('.')

print(f"DIB bars dir exists: {BARS_ROOT.exists()}")
print(f"Watchlist exists: {WATCHLIST.exists()}")
print(f"Output dir: {OUTPUT_DIR.absolute()}")

## 1. Cargar Watchlist con Eventos

In [ ]:
# Cargar todos los watchlists particionados por fecha
watchlist_files = list(WATCHLIST.rglob('watchlist.parquet'))
print(f"Encontrados {len(watchlist_files):,} watchlist files")

wl_parts = []
for wl_file in watchlist_files:
    # Extract date from path: date=YYYY-MM-DD/watchlist.parquet
    date_str = wl_file.parent.name.split('=')[1]
    df = pl.read_parquet(wl_file)
    df = df.with_columns([pl.lit(date_str).alias('date')])
    wl_parts.append(df)

wl = pl.concat(wl_parts)
print(f"Total watchlist rows: {wl.height:,}")

# Convertir date a pl.Date
wl = wl.with_columns([
    pl.col('date').str.strptime(pl.Date, format='%Y-%m-%d')
])

# Expandir una fila por evento
wl_expanded = wl.explode('events').rename({'events': 'event_code'})
print(f"Total event occurrences: {wl_expanded.height:,}")

# Eventos disponibles
events_available = sorted(wl_expanded['event_code'].unique().to_list())
print(f"\nEventos disponibles: {events_available}")

wl_expanded.head()

## 2. Funciones de Información Mutua

In [ ]:
def load_dib_bars_day(ticker: str, day: datetime.date) -> pl.DataFrame:
    """
    Carga DIB bars de un ticker en un día específico.
    """
    bars_file = BARS_ROOT / ticker / f"date={day.isoformat()}" / "dollar_imbalance.parquet"
    if not bars_file.exists():
        return None
    return pl.read_parquet(bars_file)


def aggregate_day_features(df_bars: pl.DataFrame) -> dict:
    """
    Agrega features intradía de DIB bars a features diarias.
    Solo usa columnas básicas: o, h, l, c, v, n, dollar, imbalance_score
    """
    if df_bars is None or df_bars.height == 0:
        return None
    
    # Calcular features agregados del día
    agg = df_bars.select([
        ((pl.col('c') - pl.col('o')) / pl.col('o')).mean().alias('ret_day'),
        ((pl.col('h') - pl.col('l')) / pl.col('o')).mean().alias('range_day'),
        pl.col('v').sum().alias('vol_day'),
        pl.col('dollar').sum().alias('dollar_day'),
        pl.col('imbalance_score').mean().alias('imb_day'),
        pl.col('n').sum().alias('n_bars')
    ])
    
    return agg.to_dicts()[0] if agg.height > 0 else None


def calculate_mutual_information_discretized(
    X: np.ndarray,
    y: np.ndarray,
    bins: int = 10
) -> float:
    """
    Calcula mutual information promedio entre features X y target y.
    """
    y_binned = pd.cut(y, bins=bins, labels=False, duplicates='drop')
    
    mi_scores = []
    for col_idx in range(X.shape[1]):
        x_col = X[:, col_idx]
        x_binned = pd.cut(x_col, bins=bins, labels=False, duplicates='drop')
        
        valid_mask = ~(pd.isna(x_binned) | pd.isna(y_binned))
        if valid_mask.sum() > 10:
            mi = mutual_info_score(x_binned[valid_mask], y_binned[valid_mask])
            mi_scores.append(mi)
    
    return np.mean(mi_scores) if mi_scores else 0.0


print("✓ Funciones de información mutua definidas")

## 3. Calcular MI por Día Relativo

In [ ]:
def analyze_information_by_relative_day(
    event_code: str,
    max_pre: int = 7,
    max_post: int = 7,
    sample_size: int = 500
) -> dict:
    """
    Para un evento, calcula I(X_t; y) para cada día t relativo al evento.
    
    Returns:
        {rel_day: mutual_information_score}
    """
    # Filtrar eventos de este tipo
    subset = wl_expanded.filter(pl.col('event_code') == event_code)
    
    # Sample para acelerar (opcional)
    if subset.height > sample_size:
        subset = subset.sample(sample_size, seed=42)
    
    print(f"\nAnalizando {event_code}: {subset.height} ocurrencias")
    
    # Recolectar datos por día relativo
    data_by_day = {}
    
    for rel_day in range(-max_pre, max_post + 1):
        features_list = []
        targets_list = []
        
        for row in subset.iter_rows(named=True):
            ticker = row['ticker']
            t0 = row['date']
            
            # Día relativo actual
            d = t0 + timedelta(days=rel_day)
            bars = load_dib_bars_day(ticker, d)
            
            if bars is None or bars.height == 0:
                continue
            
            # Features agregados del día
            feat = aggregate_day_features(bars)
            if feat is None:
                continue
            
            # Target: retorno futuro desde t0 (día evento)
            # Usamos bars del día t0+1, t0+2, t0+3 para calcular ret_3d
            bars_t0 = load_dib_bars_day(ticker, t0)
            bars_t3 = load_dib_bars_day(ticker, t0 + timedelta(days=3))
            
            if bars_t0 is None or bars_t3 is None:
                continue
            if bars_t0.height == 0 or bars_t3.height == 0:
                continue
            
            # Calcular retorno 3d
            p0 = bars_t0['c'][-1]
            p3 = bars_t3['c'][-1]
            ret_3d = (p3 - p0) / p0
            
            features_list.append(list(feat.values()))
            targets_list.append(ret_3d)
        
        if len(features_list) < 50:
            data_by_day[rel_day] = 0.0
            continue
        
        X = np.array(features_list)
        y = np.array(targets_list)
        
        # Calcular MI
        mi = calculate_mutual_information_discretized(X, y, bins=10)
        data_by_day[rel_day] = mi
        
        print(f"  t={rel_day:+d}: MI={mi:.4f} (n={len(features_list)})")
    
    return data_by_day


print("✓ Función de análisis por día relativo definida")

## 4. Ejecutar Análisis Information Theory

**NOTA**: Ajusta `EVENTS_TO_TEST` según necesites:
- `[:3]` → Prueba rápida (3 eventos, ~10-15 min)
- Sin slice → Análisis completo (11 eventos, ~40-60 min)

In [ ]:
# CONFIGURACIÓN: Ajusta aquí el subset de eventos
EVENTS_TO_TEST = events_available[:3]  # Cambiar a events_available para análisis completo
MAX_PRE_DAYS = 3
MAX_POST_DAYS = 3
SAMPLE_SIZE = 200  # Reducir a 100 para más velocidad, aumentar a 500 para más precisión

print(f"Analizando {len(EVENTS_TO_TEST)} eventos con ventana [{-MAX_PRE_DAYS}, {MAX_POST_DAYS}]")
print(f"Sample size: {SAMPLE_SIZE} ocurrencias por evento\n")

info_results = {}

for event in EVENTS_TO_TEST:
    info_by_day = analyze_information_by_relative_day(
        event,
        max_pre=MAX_PRE_DAYS,
        max_post=MAX_POST_DAYS,
        sample_size=SAMPLE_SIZE
    )
    info_results[event] = info_by_day

print("\n" + "="*60)
print("✓ Análisis Information Theory completado")
print(f"Eventos analizados: {len(info_results)}")
print("="*60)

## 5. Visualizar Información por Día

In [ ]:
fig, axes = plt.subplots(len(info_results), 1, figsize=(12, 4 * len(info_results)))

if len(info_results) == 1:
    axes = [axes]

for idx, (event, info_by_day) in enumerate(info_results.items()):
    ax = axes[idx]
    
    days = sorted(info_by_day.keys())
    mi_scores = [info_by_day[d] for d in days]
    
    # Normalizar
    max_mi = max(mi_scores) if max(mi_scores) > 0 else 1.0
    mi_norm = [m / max_mi for m in mi_scores]
    
    # Plot
    ax.bar(days, mi_norm, alpha=0.7, color='steelblue')
    ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Día Evento (t=0)')
    ax.axhline(y=0.1, color='orange', linestyle=':', label='Threshold 10%')
    
    # Marcar días significativos
    significant_days = [d for d, mi in zip(days, mi_norm) if mi >= 0.1]
    if significant_days:
        t_start, t_end = min(significant_days), max(significant_days)
        ax.axvspan(t_start - 0.5, t_end + 0.5, alpha=0.2, color='green',
                   label=f'Ventana sugerida: [{t_start}, {t_end}]')
    
    ax.set_xlabel('Días Relativos al Evento')
    ax.set_ylabel('Mutual Information (normalizado)')
    ax.set_title(f'{event}: Información por Día Relativo')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('information_by_day_phase1.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado: information_by_day_phase1.png")

## 6. Resumen: Ventanas Sugeridas por MI

In [ ]:
print("\n" + "="*80)
print("VENTANAS SUGERIDAS POR MUTUAL INFORMATION (threshold 10%)")
print("="*80)

suggested_windows = {}

for event, info_by_day in info_results.items():
    days = sorted(info_by_day.keys())
    mi_scores = [info_by_day[d] for d in days]
    max_mi = max(mi_scores) if max(mi_scores) > 0 else 1.0
    mi_norm = [m / max_mi for m in mi_scores]
    significant_days = [d for d, mi in zip(days, mi_norm) if mi >= 0.1]
    
    if significant_days:
        window = (min(significant_days), max(significant_days))
        suggested_windows[event] = window
        print(f"  {event:<25} → [{window[0]:+d}, {window[1]:+d}]")
    else:
        suggested_windows[event] = None
        print(f"  {event:<25} → Sin ventana clara (MI muy bajo)")

print("="*80)

## 7. Guardar Resultados Fase 1

**Output**: `phase1_results.pkl` con todos los datos necesarios para Fase 2

In [ ]:
# Empaquetar resultados
results_phase1 = {
    'info_results': info_results,
    'wl_expanded': wl_expanded,
    'events_available': events_available,
    'suggested_windows': suggested_windows,
    'config': {
        'max_pre_days': MAX_PRE_DAYS,
        'max_post_days': MAX_POST_DAYS,
        'sample_size': SAMPLE_SIZE,
        'events_tested': EVENTS_TO_TEST
    }
}

# Guardar a disco
output_file = OUTPUT_DIR / 'phase1_results.pkl'
with open(output_file, 'wb') as f:
    pickle.dump(results_phase1, f)

print("\n" + "="*80)
print("✓ FASE 1 COMPLETADA")
print("="*80)
print(f"Resultados guardados en: {output_file.absolute()}")
print(f"\nContenido:")
print(f"  - info_results: {len(info_results)} eventos con MI por día relativo")
print(f"  - wl_expanded: {wl_expanded.height:,} event occurrences")
print(f"  - events_available: {len(events_available)} eventos totales")
print(f"  - suggested_windows: {len([w for w in suggested_windows.values() if w])} ventanas sugeridas")
print(f"\nPróximo paso: Ejecutar phase2_model_performance.ipynb")
print("="*80)

In [ ]:
def export_events_for_tradingview(event_code: str, output_dir: Path = Path('.')):
    """
    Exporta eventos con timestamps exactos para visualización en TradingView.
    
    Para cada evento crea CSV con:
    - ticker
    - datetime (timestamp exacto del primer bar del evento)
    - close_price (precio en el momento del evento)
    - event_code
    - window_suggested (ventana sugerida por MI)
    """
    # Filtrar eventos de este tipo
    subset = wl_expanded.filter(pl.col('event_code') == event_code)
    
    print(f"\nProcesando {event_code}: {subset.height:,} ocurrencias")
    
    events_data = []
    
    for idx, row in enumerate(subset.iter_rows(named=True)):
        ticker = row['ticker']
        event_date = row['date']
        
        # Cargar DIB bars del día del evento para obtener timestamp exacto
        bars = load_dib_bars_day(ticker, event_date)
        
        if bars is None or bars.height == 0:
            continue
        
        # Timestamp del PRIMER bar del evento (inicio de la sesión)
        first_ts = bars['t_open'][0]  # Timestamp de apertura del primer bar
        close_price = bars['c'][0]  # Precio de cierre del primer bar
        
        # Ventana sugerida
        window = suggested_windows.get(event_code, None)
        window_str = f"[{window[0]:+d},{window[1]:+d}]" if window else "N/A"
        
        events_data.append({
            'ticker': ticker,
            'datetime': first_ts,
            'close_price': close_price,
            'event_code': event_code,
            'window_suggested': window_str,
            'date': event_date
        })
        
        # Progress cada 100 eventos
        if (idx + 1) % 100 == 0:
            print(f"  Procesados {idx + 1:,} / {subset.height:,} eventos...")
    
    if not events_data:
        print(f"⚠️  No se encontraron datos para {event_code}")
        return None
    
    # Crear DataFrame
    df = pd.DataFrame(events_data)
    
    # Ordenar por datetime
    df = df.sort_values('datetime')
    
    # Exportar CSV
    output_file = output_dir / f'tradingview_{event_code}.csv'
    df.to_csv(output_file, index=False)
    
    print(f"✓ Exportado: {output_file}")
    print(f"  Total eventos: {len(df):,}")
    print(f"  Tickers únicos: {df['ticker'].nunique()}")
    print(f"  Rango fechas: {df['date'].min()} a {df['date'].max()}")
    print(f"  Ventana sugerida: {window_str}")
    
    return df


# Crear directorio para exports
TRADINGVIEW_DIR = OUTPUT_DIR / 'tradingview_exports'
TRADINGVIEW_DIR.mkdir(exist_ok=True)

print("="*80)
print("EXPORTANDO EVENTOS PARA TRADINGVIEW")
print("="*80)

all_exports = {}

# Exportar TODOS los eventos disponibles (no solo los analizados)
for event in events_available:
    df = export_events_for_tradingview(event, TRADINGVIEW_DIR)
    if df is not None:
        all_exports[event] = df

print("\n" + "="*80)
print("✓ EXPORTACIÓN COMPLETADA")
print("="*80)
print(f"Directorio: {TRADINGVIEW_DIR.absolute()}")
print(f"Archivos generados: {len(all_exports)}")
print(f"\nEventos exportados:")
for event, df in all_exports.items():
    print(f"  • {event:<30} → {len(df):>6,} ocurrencias")
print("="*80)

print("\n📊 CÓMO USAR EN TRADINGVIEW:")
print("1. Abre TradingView con el ticker que quieres analizar")
print("2. Importa el CSV correspondiente como 'Custom Indicator'")
print("3. Los eventos aparecerán como marcadores en el gráfico")
print("4. Filtra por ticker si el CSV contiene múltiples símbolos")
print("\nEjemplo: Para ver E10_FirstGreenBounce en ticker AAPL:")
print(f"  → Carga: {TRADINGVIEW_DIR / 'tradingview_E10_FirstGreenBounce.csv'}")
print("  → Filtra columna 'ticker' = 'AAPL'")

## 10. EXPORTAR EVENTOS PARA TRADINGVIEW

**Objetivo**: Generar CSV con timestamps exactos de TODOS los eventos para visualización en TradingView.

**Formato TradingView**:
- Necesita timestamp exacto (fecha + hora)
- Un archivo por evento
- Columnas: ticker, datetime, event_code, price_at_event

Esto te permitirá:
1. Cargar los eventos como overlays en TradingView
2. Verificar visualmente cada evento sobre el gráfico de precio
3. Validar que la detección fue correcta

## 9. INTERPRETACIÓN: ¿Son creíbles los resultados?

**Criterios de validación:**

1. **Captura de rango > 70%**: La ventana debe contener la mayor parte del movimiento
2. **Volatilidad ratio > 1.5x**: Dentro de ventana debe haber más volatilidad que fuera
3. **Consistencia visual**: Los gráficos deben mostrar patrones claros

**Posibles problemas detectados:**
- Si todas las ventanas son [-3, +3] → Threshold demasiado bajo, MI no discrimina
- Si captura < 50% → Ventana no útil para trading
- Si vol_ratio < 1.0 → Ventana captura el período tranquilo (ERROR)

**Próximos pasos si resultados no son creíbles:**
1. Ajustar threshold de MI (probar 30%, 50% en lugar de 10%)
2. Usar MI absoluto en lugar de normalizado
3. Comparar con Phase 2 (LightGBM) para validación cruzada

In [ ]:
def get_daily_closes(ticker: str, start_date, end_date):
    """
    Obtiene precios de cierre diarios para un rango de fechas.
    Devuelve: list of (date, close_price)
    """
    prices = []
    current = start_date
    
    while current <= end_date:
        bars = load_dib_bars_day(ticker, current)
        if bars is not None and bars.height > 0:
            close_price = bars['c'][-1]  # Último cierre del día
            prices.append((current, close_price))
        current += timedelta(days=1)
    
    return prices


def visualize_window_capture(event_code: str, window: tuple, n_examples: int = 6):
    """
    Visualiza ejemplos reales de eventos con ventana superpuesta.
    Calcula % de movimiento capturado.
    """
    if window is None:
        print(f"❌ {event_code}: Sin ventana sugerida")
        return
    
    pre, post = window
    
    # Obtener ocurrencias del evento
    subset = wl_expanded.filter(pl.col('event_code') == event_code)
    
    # Tomar muestra aleatoria
    if subset.height > n_examples:
        subset = subset.sample(n_examples, seed=123)
    else:
        n_examples = subset.height
    
    # Configurar grid de subplots
    n_cols = 3
    n_rows = (n_examples + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    axes = axes.flatten() if n_examples > 1 else [axes]
    
    capture_stats = []
    
    for idx, row in enumerate(subset.iter_rows(named=True)):
        if idx >= n_examples:
            break
            
        ticker = row['ticker']
        t0 = row['date']
        
        # Cargar precios desde t-5 hasta t+5 (ventana amplia para contexto)
        start = t0 - timedelta(days=5)
        end = t0 + timedelta(days=5)
        prices = get_daily_closes(ticker, start, end)
        
        if len(prices) < 5:  # Muy pocos datos
            continue
        
        # Separar fechas y precios
        dates = [p[0] for p in prices]
        closes = [p[1] for p in prices]
        
        # Calcular días relativos al evento
        rel_days = [(d - t0).days for d in dates]
        
        # Identificar puntos dentro de ventana sugerida
        in_window = [(pre <= rd <= post) for rd in rel_days]
        
        # Calcular captura de movimiento
        all_prices = closes
        window_prices = [p for p, in_w in zip(closes, in_window) if in_w]
        
        if len(window_prices) < 2 or len(all_prices) < 2:
            continue
        
        # Rango total vs rango capturado
        total_range = max(all_prices) - min(all_prices)
        window_range = max(window_prices) - min(window_prices)
        capture_pct = (window_range / total_range * 100) if total_range > 0 else 0
        
        # Volatilidad dentro vs fuera
        prices_in = [p for p, in_w in zip(closes, in_window) if in_w]
        prices_out = [p for p, in_w in zip(closes, in_window) if not in_w]
        
        vol_in = np.std(np.diff(prices_in)) if len(prices_in) > 1 else 0
        vol_out = np.std(np.diff(prices_out)) if len(prices_out) > 1 else 0
        vol_ratio = vol_in / vol_out if vol_out > 0 else np.nan
        
        capture_stats.append({
            'ticker': ticker,
            'date': t0,
            'capture_pct': capture_pct,
            'vol_ratio': vol_ratio
        })
        
        # Graficar
        ax = axes[idx]
        
        # Precios fuera de ventana (gris)
        out_days = [rd for rd, in_w in zip(rel_days, in_window) if not in_w]
        out_prices = [p for p, in_w in zip(closes, in_window) if not in_w]
        ax.plot(out_days, out_prices, 'o-', color='gray', alpha=0.4, label='Fuera ventana')
        
        # Precios dentro de ventana (verde/rojo)
        in_days = [rd for rd, in_w in zip(rel_days, in_window) if in_w]
        in_prices = [p for p, in_w in zip(closes, in_window) if in_w]
        ax.plot(in_days, in_prices, 'o-', color='green', linewidth=2, markersize=8, label='Dentro ventana')
        
        # Marcar día evento
        ax.axvline(x=0, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Evento')
        
        # Sombrear ventana
        ax.axvspan(pre, post, alpha=0.15, color='green')
        
        # Título con estadísticas
        ax.set_title(f"{ticker} ({t0})\nCaptura: {capture_pct:.1f}% | Vol ratio: {vol_ratio:.2f}x", 
                     fontsize=10)
        ax.set_xlabel('Días relativos al evento')
        ax.set_ylabel('Precio ($)')
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=8)
    
    # Ocultar axes vacíos
    for idx in range(len(subset), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig(f'validation_{event_code}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Estadísticas agregadas
    if capture_stats:
        df_stats = pd.DataFrame(capture_stats)
        print(f"\n{'='*70}")
        print(f"ESTADÍSTICAS DE CAPTURA: {event_code}")
        print(f"Ventana sugerida: [{pre:+d}, {post:+d}]")
        print(f"{'='*70}")
        print(f"  Captura de rango (%):")
        print(f"    - Media:   {df_stats['capture_pct'].mean():.1f}%")
        print(f"    - Mediana: {df_stats['capture_pct'].median():.1f}%")
        print(f"    - Min:     {df_stats['capture_pct'].min():.1f}%")
        print(f"    - Max:     {df_stats['capture_pct'].max():.1f}%")
        print(f"\n  Volatilidad dentro/fuera:")
        print(f"    - Media:   {df_stats['vol_ratio'].mean():.2f}x")
        print(f"    - Mediana: {df_stats['vol_ratio'].median():.2f}x")
        print(f"{'='*70}")
        print(f"\n✓ Gráfico guardado: validation_{event_code}.png\n")
    
    return capture_stats


# Ejecutar validación para cada evento
print("\n" + "="*80)
print("VALIDACIÓN VISUAL: Captura de Movimiento Real")
print("="*80)

all_stats = {}
for event, window in suggested_windows.items():
    stats = visualize_window_capture(event, window, n_examples=6)
    all_stats[event] = stats

print("\n" + "="*80)
print("✓ Validación visual completada")
print("="*80)

## 8. VALIDACIÓN VISUAL: ¿Las ventanas capturan realmente el movimiento?

**CRÍTICO**: Necesitamos VERIFICAR que las ventanas sugeridas realmente capturan el movimiento de precio, no solo confiar en MI.

Este análisis muestra:
1. Gráficos de precio real con ventana superpuesta
2. % del movimiento total capturado por la ventana
3. Comparación de volatilidad dentro vs fuera de ventana
4. Ejemplos aleatorios para inspección visual